# COBRAme Reaction Buidling

In [1]:
import cobrame
from cobrame.util import dogma, building
import cobra
import cobra.test
from collections import defaultdict

#import warnings
#warnings.filterwarnings('ignore')

## Overview

COBRAme is constructed entirely over COBRApy. This means that ME reactions will have all of the same properties, methods, and functions as a COBRApy reaction. However, one key difference is that, in order to facilliate the template nature of many gene expression reactions, reactions are constructed and their components are manipulated via the use of the ProcessData classes. These act as information vessels for holding the information assocatied with a cellular process in simple, standard datatypes such as dictionaries and strings. 

-----

This tutorial will go step-by-step through the process of creating the generic enzyme catalyzed reaction or metabolic reaction:
$$ a \rightarrow b $$
that requires the formation of **complex\_ab** in order to proceed

-------

In order for this reaction to carry flux in the model we will additionally need to first add the corresponding:
1. Transcription reactions
2. Translation reactions
3. tRNA charging reactions
4. Complex formation reactions

Once these are added we will add in synthesis of key macromolecular components (ribosome, RNA polymerase, etc.) and show how they are applied to their respective reactions with their coupling constraints implemented.

## Initializing new ME-models

When applying some constraints in the ME-model, metabolite properties are required. For instance, to calculate the total biomass produced by a particular reaction, the amino acid, nucleotide, etc. molecular weights are required.To enable these calculations, all metabolites from iJO1366, along with their metabolite attributes are added to the newly created ME-model.

Further the reactions from *i*JO1366 will be added to the ME-model to demonstrate ME-model solving procedures.

In [2]:
# create empty ME-model
me = cobrame.MEmodel('test')
ijo = cobra.test.create_test_model('ecoli')

# Add all metabolites and reactions from iJO1366 to the new ME-model
for met in ijo.metabolites:
    me.add_metabolites(met)
for rxn in ijo.reactions:
    me.add_reaction(rxn)

The ME-model contains a "global_info" attribute which stores information used to calculate coupling constraints, along with other functions. The specifics of each of these constraints will be discussed when they are implemented.

*Note*: k_deg will initially be set to 0. We will apply RNA degradation later in the tutorial.

In [3]:
# "Translational capacity" of organism
me.global_info['kt'] =  4.5 #(in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] =  0.087 #scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] =  0. #1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1700. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000. #109. / 1000. # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000. # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000. # in kDA
me.global_info['f_tRNA'] = .12

sequence = ("ATG" + "TTT" * 12 + "TAT" * 12 + 
            "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA")

## Add Transcription

The general workflow for adding any reaction to a ME-model using COBRAme occurs in three steps:
1. Create the ProcessData(s) associated with the reaction and populate them with the necessary information
2. Create the MEReaction and link the appropriate ProcessData
3. Execute the MEReaction's update method

### Without utility functions

In [4]:
data = cobrame.TranscriptionData('TU_a',me,RNA_products={'RNA_a'})
data.nucleotide_sequence = sequence
rxn = cobrame.TranscriptionReaction('transcription_TU_a')
rxn.transcription_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 86 amp_c + 38 cmp_c + 12 gmp_c + 186 h_c + 187 ppi_c + 50 ump_c


/home/sbrg-cjlloyd/cobrame/cobrame/core/MEReactions.py:505 UserWarning: RNA Polymerase () not found
/home/sbrg-cjlloyd/cobrame/cobrame/core/MEReactions.py:516 UserWarning: Transcript (RNA_a) added to model 


As stated in the above warning, we have not added the RNA_a metabolite to the model. This means that the model does not know the RNA_type (mRNA, tRNA or rRNA) or the specific sequence of RNA_a. Therefore, the *biomass* constraint for this RNA cannot be included in the reaction since the sequence is required to calculate the molecular weight of RNA_a. This sequence cannot be determined from the transcription unit (TU) sequence because a single TU often contains several different RNAs. Adding this information to the RNA_a metabolite and updating the transcription reaction adds the *biomass* constraint to the reaction. 

**Note**: the RNA_polymerase complex is not included in the reaction. This will be added later 

In [5]:
me.metabolites.RNA_a.RNA_type = 'mRNA'
me.metabolites.RNA_a.nucleotide_sequence = sequence
rxn.update()
print rxn.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 186 h_c + 59.172286 mRNA_biomass + 186 ppi_c


This reaction also produces a small amount of the a $biomass$ metabolite (constraint). This term has a coefficient corresponding to the molecular weight (in $kDA$) of the protein being translated. This constraint will be implemented into a $v_{biomasss\_dilution}$ reaction with the form: 
$$\mu \leq v_{biomass\_dilution} \leq \mu $$

A mathematical description of the *biomass* constraint can be found in **Biomass Dilution Constraints**.

### Using utility functions
Using the utility functions to create the TranscribedGene metabolite has the advantage of forcing the assignment of sequence, strand and RNA_type. 

In [6]:
building.create_transcribed_gene(me, 'b', 1, 2, sequence, '+', 'mRNA')
building.add_transcription_reaction(me, 'TU_b', {'b'}, sequence)
print me.reactions.transcription_TU_b.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_b + 186 h_c + 59.172286 mRNA_biomass + 186 ppi_c


**Note**: This is not a complete picture of transcriptiont because the RNA polymerase is missing.
### Associate RNA Polymerase

In [7]:
RNAP = cobrame.RNAP('RNA_polymerase')
me.add_metabolites(RNAP)
for data in me.transcription_data:
    data.RNA_polymerase = RNAP.id
me.reactions.transcription_TU_a.update()
me.reactions.transcription_TU_b.update()

print(me.reactions.transcription_TU_a.reaction)

0.000759722875816993*mu + 0.000297431505882353 RNA_polymerase + 86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 186 h_c + 59.172286 mRNA_biomass + 186 ppi_c


The coefficient for RNA_polymerase is the first instance where a coupling constraint is imposed. In this case the constraint couples the formation of a RNA_polymerase metabolite to its transcription flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808), with assumption that $k_{rnap} = 3 \cdot k_{ribosome}$ based on data from [Proshkin et al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/20413502):

$$
\begin{align}
v_{dilution,RNAP, j} =  \frac{l_{TU,j}}{3 c_{ribo}\kappa_{\tau}} v_{transcription,j} (\mu+r_0\kappa_{\tau}), & \forall j \in TU
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{ribo} = \frac{m_{rr}}{f_{rRNA}\cdot m_{aa}}$ where: $m_{nt}$ is the mass of rRNA per ribosome. $f_{rRNA}$ is the fraction of total RNA that is rRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{transcription, j}$ is the rate of transcription for $TU_j$
 
 - $l_{TU, j}$ is number of nucleotides in $TU_j$
 
----
## Add Translation
### Without utility functions
#### Add the TranslationData for protein_a
In order to add a TranslationData object to a ME model the user must additionally specifify the mRNA id and protein id of the translation reaction that will be added. This information as well as a nucleotide sequence is the only information required to add a translation reaction.

In [8]:
data = cobrame.TranslationData('a', me, 'RNA_a', 'protein_a')
data.nucleotide_sequence = sequence

#### Add the TranslationReaction for protein_a
By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a reaction reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [9]:
rxn = cobrame.TranslationReaction('translation_a')
rxn.translation_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

/home/sbrg-cjlloyd/cobrame/cobrame/core/MEReactions.py:652 UserWarning: ribosome not found
/home/sbrg-cjlloyd/cobrame/cobrame/core/MEReactions.py:692 UserWarning: RNA_degradosome not found


0.00149519890260631*mu + 0.00058537037037037 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass


In this case the constraint couples the formation of a mRNA metabolite to its translation flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808):

$$
\begin{align}
& v_{dilution,j} = \frac{3}{\kappa_{\tau} c_{mRNA}} \cdot (\mu + \kappa_{\tau} r_0) v_{translation,j}  , & \forall j \in mRNA \\
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{mRNA} = \frac{m_{nt}}{f_{mRNA}\cdot m_{aa}}$ where: $m_{nt}$ is the molecular weight of an average mRNA nucleotide. $f_{mRNA}$ is the fraction of total RNA that is mRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{translation, j}$ is the rate of translation for $mRNA_j$


### Using utility functions
`add_translation_reaction` assumes that the RNA and protein have the same locus_id. It creates the appropriate `TranslationData` and `TranslationReaction` instance, links the two together and updates the `TranslationReaction`.


In [10]:
building.add_translation_reaction(me, 'b', dna_sequence=sequence, update=True)
print me.reactions.translation_b.reaction

0.00149519890260631*mu + 0.00058537037037037 RNA_b + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_b + 7.500606 protein_biomass


### Incorporate Ribosome

In [11]:
ribosome = cobrame.Ribosome('ribosome')
me.add_metabolites([ribosome])
me.reactions.translation_a.update()
print me.reactions.translation_a.reaction

0.00149519890260631*mu + 0.00058537037037037 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 0.000747469281045752*mu + 0.000292634223529412 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass


This imposes a new coupling constraint for the ribosome. In this case the constraint couples the formation of a ribosome to its translation flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808):

$$
\begin{align}
v_{dilution,ribo, j} = \frac{l_{p,j}}{c_{ribo}\kappa_{\tau}} v_{translation,j} (\mu+r_0\kappa_{\tau}) , & \forall j \in mRNA
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{ribo} = \frac{m_{rr}}{f_{rRNA}\cdot m_{aa}}$ where: $m_{nt}$ is the mass of rRNA per ribosome. $f_{rRNA}$ is the fraction of total RNA that is rRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{translation, j}$ is the rate of translation for $mRNA_j$
 
 - $l_{p, j}$ is number of amino acids in peptide translated from $mRNA_j$
 
-------
**Note**: The above reactions do not provide a complete picture of translation considering that it's missing charged tRNAs to facillitate tRNA addition.

Below, we'll correct this by adding in an tRNA charging reaction.

## Add tRNA charging reaction

### Without utility functions (There are no utility functions for adding individual tRNA charging reactions)
#### Add tRNAData

In [12]:
data = cobrame.tRNAData('tRNA_d_GUA', me, 'val__L_c', 'RNA_d', 'GUA')
rxn = cobrame.tRNAChargingReaction('charging_tRNA_d_GUA')
me.add_reaction(rxn)
rxn.tRNAData = data

#### Add tRNAChargingReaction

In [13]:
#Setting verbose=False suppresses print statements indicating that new metabolites were created
rxn.update(verbose=False)
print(rxn.reaction)

0.000116266666666667*mu + 4.55184e-5 RNA_d + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


This reaction creates one ```generic_charged_tRNA``` equivalement that can then be used in a translation reaction

The coefficient for ```RNA_d``` and ```lys__L_c``` are defined by:

$$
\begin{align}
v_{dilution,j} \geq \frac{1}{\kappa_{\tau} c_{tRNA,j}} (\mu + \kappa_{\tau} r_0)  v_{charging,j} , & \forall j \in tRNA
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{tRNA, j} = \frac{m_{tRNA}}{f_{tRNA}\cdot m_{aa}}$ where: $m_{tRNA}$ is molecular weight of an average tRNA. $f_{tRNA}$ is the fraction of total RNA that is tRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{charging, j}$ is the rate of charging for $tRNA_j$
 

**Note**: This tRNA charging reaction is still missing a tRNA synthetase which catalyzes the amino acid addition to the uncharged tRNA.

### Incorporate tRNA Synthetases

In [14]:
synthetase = cobrame.Complex('synthetase')
me.add_metabolites(synthetase)
data.synthetase = synthetase.id
rxn.update()
print(rxn.reaction)

0.000116266666666667*mu + 4.55184e-5 RNA_d + 4.27350427350427e-6*mu*(0.000116266666666667*mu + 1.0000455184) synthetase + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


The synthetase coupling was reformulated from [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808) enable more modularity in the ME-model. A more complete mathematical description of the tRNA synthetase coupling constraints can be found in the [tRNA.ipynb](tRNA.ipynb)

---

### Add tRNAs to translation

Here we take advantage of an additional subclass of `ProcessData`, called a `SubreactionData` object. This class is used to lump together processeses that occur as a result of many individual reactions, including translation elongation, ribosome formation, tRNA modification, etc. Since each of these steps often involve an enzyme that requires its own coupling constraint, this process allows these processes to be lumped into one reaction while still enabling each subprocess to be modified.

`TranslationData` objects have an `elongation_subreactions` method that returns any subreactions that have been used added to the model and are part of translation elongation. Since no tRNA-mediated amino acid addition subreactions have been added to the model, the below call returns nothing.

In [15]:
print me.translation_data.a.elongation_subreactions

{}


/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:318 UserWarning: RNA_a starts with 'AUG' which is not a start codon
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction thr_addition_at_ACG not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction met_addition_at_AUG not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction phe_addition_at_UUU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction asp_addition_at_GAU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction tyr_addition_at_UAU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/ProcessData.py:356 UserWarning: elongation subreaction ser_addition_at_AGU not in model


`UserWarnings` are returned to indicate that tRNA subreactions have not been added for each codon.

Below, we add the SubreactionData (excluding enzymes) for the addition of an amino acid using information from the *E. coli* codon table.

In [16]:
data = cobrame.SubreactionData('asp_addition_at_GAU', me)
data.stoichiometry = {'generic_tRNA_GAU_asp__L_c': -1,
                      'gtp_c': -1, 'gdp_c': 1, 'h_c': 1,
                      'pi_c': 1}

Now calling `elongation_subreactions` returns the number of tRNA subreactions that should be added to the `TranslationData`

In [17]:
translation_subreactions = me.translation_data.a.elongation_subreactions
print translation_subreactions

{'asp_addition_at_GAU': 12}


Updating `TranslationData.subreactions` with the tRNA subreactions incorporates this information into the `TranslationReaction`

In [18]:
print("Before adding tRNA subreaction")
print("------------------------------")
print(me.reactions.translation_a.reaction)
print("")
# Update tranlation_data with subreactions and update
for subreaction, value in translation_subreactions.items():
    me.translation_data.a.subreactions[subreaction] = value
me.reactions.translation_a.update(verbose=False)
print("After adding tRNA subreaction")
print("-----------------------------")
print(me.reactions.translation_a.reaction)

Before adding tRNA subreaction
------------------------------
0.00149519890260631*mu + 0.00058537037037037 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 0.000747469281045752*mu + 0.000292634223529412 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass

After adding tRNA subreaction
-----------------------------
0.00149519890260631*mu + 0.00058537037037037 RNA_a + 12 asp__L_c + 12 generic_tRNA_GAU_asp__L_c + 12 gtp_c + met__L_c + 12 phe__L_c + 0.000747469281045752*mu + 0.000292634223529412 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 12 gdp_c + 12 h_c + 12 pi_c + protein_a + 7.500606 protein_biomass


## Create complexes
### Without utility functions
#### Create ComplexData and ComplexFormation
For COBRAme models, the reaction gene-protein-reaction rule (GPR) is replaced with a metabolite representing the synthesis of the enzyme(s) catalyzing a reaction. This metabolite is formed explicitly in a ME model by seperate reaction to transcribe the gene(s) and translate the protein(s) the compose the complex. These reactions will be added later.

ComplexData objects contain:
 - Subunit stoichiometry
 - Modification dictionary (discussed later)
 - Translocation dictionary (discussed later)

In [19]:
data = cobrame.ComplexData('complex_ab', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}

rxn = cobrame.ComplexFormation('formation_complex_ab')
me.add_reaction(rxn)
rxn.complex_data_id = data.id
rxn._complex_id = data.id
rxn.update(verbose=False)
print me.reactions.formation_complex_ab.reaction

protein_a + protein_b --> complex_ab


### Using utility functions
Alternatively, ComplexData has a ```create_complex_formation()``` function to create the sythesis reaction following the naming conventions. It contains an ```update()``` function which incorporates changes in the ComplexData

In [20]:
data = cobrame.ComplexData('complex_ba', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}
data.create_complex_formation()
print me.reactions.formation_complex_ba.reaction

protein_a + protein_b --> complex_ba


### Apply modification to complex formation reaction
Many enzyme complexes in an ME-model require cofactors or prosthetic groups in order to properly function. Information about such processes are stored as ModificationData.

For instance, we can add the modification of an iron-sulfur cluster, a common prosthetic group, by doing the following:

In [21]:
# Define the stoichiometry of the modification 
mod_data = cobrame.ModificationData('mod_2fe2s_c', me)
mod_data.stoichiometry = {'2fe2s_c': -1}
# this process can also be catalyzed by a chaperone
mod_data.enzyme = 'complex_ba'
mod_data.keff = 65.  # default value

Associate modification to complex and ```update()``` its formation

In [22]:
complex_data = me.complex_data.complex_ab
complex_data.modifications['mod_2fe2s_c'] = 1
print me.reactions.formation_complex_ab.reaction
me.reactions.formation_complex_ab.update()
print me.reactions.formation_complex_ab.reaction

protein_a + protein_b --> complex_ab
2fe2s_c + 4.27350427350427e-6*mu complex_ba + protein_a + protein_b --> 0.17582 biomass + complex_ab


## Add Metabolic Reaction
MetabolicReactions require, at a minimum, one corresponding StoichiometricData. StoichiometricData essentially holds the information contained in an M-model reaction. This includes the metabolite stoichiometry and the upper and lower bound of the reaction. As a best practice, StoichiometricData typically uses an ID equivalent to the M-model reaction ID.

### Without utility functions
So first, we will create a StoichiometricData object to define the stoichiometry of the conversion of *a* to *b*

The code below creates a StoichiometricData object which stores all of the stoichiometric information associated with the MetabolicReaction. This includes:
 - Reaction Stoichiometry
 - Lower and upper bound
 
Only one StoichiometricData object should be created for both reversible and irreversible reactions

In [23]:
# unique to COBRAme, construct a stoichiometric data object with the reaction information
data = cobrame.StoichiometricData('a_to_b', me)
stoichiometry = {'a':-1, 'b': 1}
data._stoichiometry = stoichiometry
data.lower_bound = -1000
data.upper_bound = 1000

The StoichiometricData for this reversible reaction is then assigned to two different MetabolicReactions (Due to the enzyme dilution constraint, all enzyme catalyzed reactions must be reverisble; more on this later). The MetabolicReactions require:
 - The associated StoichiometricData
 - The *reverse* flag set to True for reverse reactions, False for forward reactions
 - Enzyme $K_{eff}$ for reaction (discussed later, dafault=65)

These fields are then processed and the actual model reaction is created using the MetabolicReaction's update() function

In [24]:
# Create a forward ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_FWD_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = False
rxn.keff = 65.
# Update
rxn.update(verbose=False)

# Create a reverse ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_REV_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = True
rxn.keff = 65.
# Update
rxn.update(verbose=False)


print me.reactions.a_to_b_FWD_complex_ab.reaction
print me.reactions.a_to_b_REV_complex_ab.reaction

a --> b
b --> a


**Note**: the $k_{eff}$ and `complex_ab` is not included in the reaction since no complex has been associated to it yet

### Associate enzyme with MetabolicReaction

The ComplexData object created in the previous cell can be incorporated into the MetabolicReaction using code below. 

**Note**: the update() function is required to apply the change.

In [25]:
data = me.complex_data.complex_ab
me.reactions.a_to_b_FWD_complex_ab.complex_data = data
print('Forward reaction (before update): %s' %
      (me.reactions.a_to_b_FWD_complex_ab.reaction))
me.reactions.a_to_b_FWD_complex_ab.update()
print('Forward reaction (after update): %s' %
      (me.reactions.a_to_b_FWD_complex_ab.reaction))
print('')

me.reactions.a_to_b_REV_complex_ab.complex_data = data
print('Reverse reaction (before update): %s' %
      (me.reactions.a_to_b_REV_complex_ab.reaction))
me.reactions.a_to_b_REV_complex_ab.update()
print('Reverse reaction (after update): %s' %
      (me.reactions.a_to_b_REV_complex_ab.reaction))

Forward reaction (before update): a --> b
Forward reaction (after update): a + 4.27350427350427e-6*mu complex_ab --> b

Reverse reaction (before update): b --> a
Reverse reaction (after update): b + 4.27350427350427e-6*mu complex_ab --> a


The coefficient for complex_ab is determined by the expression $$\frac{\mu}{k_{eff}}$$ which in its entirety represents the dilution of an enzyme following a cell doubling. The coupling constraint can be summarized as followed
$$ 
\begin{align}
&v_{dilution,j} = \mu \sum_{i} \left( \frac{1}{k_{eff,i}} v_{usage,i} \right), & \forall j \in Enzyme
\end{align}
$$

Where 

 - $v_{usage,i}$ is the flux through the metabolic reaction
 - $k_{eff}$ is the turnover rate for the process and conveys the productivity of the enzyme complex. Physically, it can be thought of as the number of reactions the enzyme can catalyze per cell division. 


By default the $k_{eff}$ for a MetabolicReaction is set to 65 but this can be changed using the code below.

In [26]:
me.reactions.a_to_b_FWD_complex_ab.keff = .00001
me.reactions.a_to_b_FWD_complex_ab.update()

# The forward and reverse direction can have differing keffs
print('Forward reaction')
print('----------------')
print(me.reactions.a_to_b_FWD_complex_ab.reaction)
print('')
print('Reverse reaction')
print('----------------')
print(me.reactions.a_to_b_REV_complex_ab.reaction)

Forward reaction
----------------
a + 27.7777777777778*mu complex_ab --> b

Reverse reaction
----------------
b + 4.27350427350427e-6*mu complex_ab --> a


### Using utility functions

In [27]:
stoich_data = cobrame.StoichiometricData('b_to_c', me)
stoich_data._stoichiometry = {'b': -1, 'c': 1}
stoich_data.lower_bound = 0
stoich_data.upper_bound = 1000.
building.add_metabolic_reaction_to_model(me, stoich_data.id, 'forward', complex_id='complex_ab',
                                         update=True)
print('Reaction b_to_c')
print('---------------')
print(me.reactions.b_to_c_FWD_complex_ab.reaction)

Created <Metabolite c at 0x7f1ef2bd1d90> in <MetabolicReaction b_to_c_FWD_complex_ab at 0x7f1ef2b63f50>
Reaction b_to_c
---------------
b + 4.27350427350427e-6*mu complex_ab --> c
